## Install the needed packages

In [4]:
#Geocode Package

import sys
!{sys.executable} -m pip install geocoder
print('package installed')

package installed


In [5]:
# install geopy package

!conda install -c conda-forge geopy --yes
print('package installed')

Solving environment: done

# All requested packages already installed.

package installed


In [6]:
# install folium package

!conda install -c conda-forge folium=0.5.0 --yes
print('package installed')

Solving environment: done

# All requested packages already installed.

package installed


In [9]:
#import libraries

import pandas as pd  # library for data analsysis
import requests  # library to handle requests
import geocoder
import json  # library to handle JSON files
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import numpy as np  # library to handle data in a vectorized manner
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values
from sklearn.cluster import KMeans  # import k-means from clustering stage
import folium  # map rendering library

In [10]:
# Get neighborhood data: 'cod_distrito', 'distrito', 'cod_barrio', 'barrio' 

neighborhoods = pd.read_csv('BARRIOS.csv',sep=';')
neighborhoods = neighborhoods.rename(columns={'Codigo de distrito al que pertenece': 'Codigo de distrito'})
neighborhoods = neighborhoods.drop(columns={'Superficie (m2)', 'Perimetro (m)'})

neighborhoods

,Codigo de barrio,Codigo de distrito,Nombre de barrio
0,1,1,PALACIO
1,1,2,IMPERIAL
2,1,3,PACIFICO
3,1,4,RECOLETOS
4,1,5,EL VISO
...,...,...,...
126,7,20,CANILLEJAS
127,8,8,EL GOLOSO
128,8,15,ATALAYA
129,8,20,EL SALVADOR


In [11]:
boroughs = pd.read_csv('DISTRITOS.csv',sep=';')
boroughs = boroughs.drop(columns={'Superficie (m2)', 'Perimetro (m)','Numero de barrios'})
boroughs

,Codigo de distrito,Nombre de distrito
0,1,CENTRO
1,2,ARGANZUELA
2,3,RETIRO
3,4,SALAMANCA
4,5,CHAMARTIN
5,6,TETUAN
6,7,CHAMBERI
7,8,FUENCARRAL-EL PARDO
8,9,MONCLOA-ARAVACA
9,10,LATINA


In [12]:
boroughs_mad = pd.merge(boroughs, neighborhoods, how="left", on=["Codigo de distrito"])
boroughs_mad = boroughs_mad.drop(columns={'Codigo de barrio'})
boroughs_mad = boroughs_mad.rename(columns={'Nombre de distrito':'borough','Nombre de barrio':'neighborhood','Codigo de distrito':'borough code'})
boroughs_mad['Latitude'] = 0.0
boroughs_mad['Longitude'] = 0.0
boroughs_mad

,borough code,borough,neighborhood,Latitude,Longitude
0,1,CENTRO,PALACIO,0.0,0.0
1,1,CENTRO,EMBAJADORES,0.0,0.0
2,1,CENTRO,CORTES,0.0,0.0
3,1,CENTRO,JUSTICIA,0.0,0.0
4,1,CENTRO,UNIVERSIDAD,0.0,0.0
...,...,...,...,...,...
126,21,BARAJAS,ALAMEDA DE OSUNA,0.0,0.0
127,21,BARAJAS,AEROPUERTO,0.0,0.0
128,21,BARAJAS,CASCO H.BARAJAS,0.0,0.0
129,21,BARAJAS,TIMON,0.0,0.0


In [13]:
# call for arcgis to assess the coordinates for each Neighborhood
for i in range(0, len(boroughs_mad)):
    dir = boroughs_mad.iloc[i,2] + ', ' + boroughs_mad.iloc[i,1] + ', madrid, españa'  # "Neighborhood, Borough, 'madrid', 'españa'"
    g = geocoder.arcgis(dir)
    boroughs_mad.iloc[i,3] = g.latlng[0]
    boroughs_mad.iloc[i,4] = g.latlng[1]
    
boroughs_mad

,borough code,borough,neighborhood,Latitude,Longitude
0,1,CENTRO,PALACIO,40.40958,-3.88009
1,1,CENTRO,EMBAJADORES,40.39152,-3.69289
2,1,CENTRO,CORTES,40.41607,-3.69893
3,1,CENTRO,JUSTICIA,40.42479,-3.69308
4,1,CENTRO,UNIVERSIDAD,40.42565,-3.70726
...,...,...,...,...,...
126,21,BARAJAS,ALAMEDA DE OSUNA,40.45818,-3.58953
127,21,BARAJAS,AEROPUERTO,40.46148,-3.57666
128,21,BARAJAS,CASCO H.BARAJAS,40.36349,-5.14696
129,21,BARAJAS,TIMON,40.47366,-3.57777


In [14]:
# Foursquare credentials
CLIENT_ID = '5PDPRZ2A2ICKWCPF40WJYJGMXP24NH1QDS3EFI2WOX1K2HV1'
CLIENT_SECRET = 'HLPXY3G4VQDS4IIKVVZUJEDN5ZRKYL4GV1KG2I0WTMPB5YVJ'
VERSION = '20201110'

In [18]:
# define the venues dataframe
venues_columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'venue', 
                  'venue Latitude', 
                  'venue Longitude', 
                  'venue Category']

madrid_venues = pd.DataFrame(columns=venues_columns)

In [25]:
# function finding venues for a given location
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'venue', 
                  'venue Latitude', 
                  'venue Longitude', 
                  'venue Category']
    
    return(nearby_venues)

In [26]:
#call the function
madrid_venues = getNearbyVenues(names=boroughs_mad['neighborhood'],
                                   latitudes=boroughs_mad['Latitude'],
                                   longitudes=boroughs_mad['Longitude'])

madrid_venues

PALACIO             
EMBAJADORES         
CORTES              
JUSTICIA            
UNIVERSIDAD         
SOL                 
IMPERIAL            
ACACIAS             
CHOPERA             
LEGAZPI             
DELICIAS            
PALOS DE MOGUER     
ATOCHA              
PACIFICO            
ADELFAS             
ESTRELLA            
IBIZA               
LOS JERONIMOS       
NI—O JESUS          
RECOLETOS           
GOYA                
FUENTE DEL BERRO    
GUINDALERA          
LISTA               
CASTELLANA          
EL VISO             
PROSPERIDAD         
CIUDAD JARDIN       
HISPANOAMERICA      
NUEVA ESPA—A        
CASTILLA            
BELLAS VISTAS       
CUATRO CAMINOS      
CASTILLEJOS         
ALMENARA            
VALDEACEDERAS       
BERRUGUETE          
GAZTAMBIDE          
ARAPILES            
TRAFALGAR           
ALMAGRO             
RIOS ROSAS          
VALLEHERMOSO        
EL PARDO            
FUENTELARREINA      
PE—A GRANDE         
EL PILAR            
LA PAZ       

,neighborhood,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
0,PALACIO,40.40958,-3.88009,Proverbium,40.408192,-3.877232,Italian Restaurant
1,PALACIO,40.40958,-3.88009,The London Walk,40.408197,-3.880682,Irish Pub
2,PALACIO,40.40958,-3.88009,Mesón Gallego O'Carro,40.405752,-3.883683,Spanish Restaurant
3,PALACIO,40.40958,-3.88009,La Barbacana,40.405809,-3.875419,Spanish Restaurant
4,PALACIO,40.40958,-3.88009,Trattoria Piccolo Mio,40.405704,-3.876092,Italian Restaurant
...,...,...,...,...,...,...,...
7245,CORRALEJOS,40.46540,-3.61164,Novotel Campo de las Naciones,40.462509,-3.615084,Hotel
7246,CORRALEJOS,40.46540,-3.61164,Telepizza,40.469282,-3.616347,Pizza Place
7247,CORRALEJOS,40.46540,-3.61164,El vínculo,40.461391,-3.616465,Restaurant
7248,CORRALEJOS,40.46540,-3.61164,Balcón de Las Naciones,40.461948,-3.621289,Paella Restaurant


In [27]:
madrid_venues.groupby('neighborhood').count()

,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
neighborhood,,,,,,
ABRANTES,39,39,39,39,39,39
ACACIAS,100,100,100,100,100,100
ADELFAS,39,39,39,39,39,39
AEROPUERTO,44,44,44,44,44,44
ALAMEDA DE OSUNA,43,43,43,43,43,43
...,...,...,...,...,...,...
VENTAS,78,78,78,78,78,78
VILLAVERDE ALTO C.H.,17,17,17,17,17,17
VINATEROS,32,32,32,32,32,32


In [29]:
madrid_venues.head()

,neighborhood,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
0,PALACIO,40.40958,-3.88009,Proverbium,40.408192,-3.877232,Italian Restaurant
1,PALACIO,40.40958,-3.88009,The London Walk,40.408197,-3.880682,Irish Pub
2,PALACIO,40.40958,-3.88009,Mesón Gallego O'Carro,40.405752,-3.883683,Spanish Restaurant
3,PALACIO,40.40958,-3.88009,La Barbacana,40.405809,-3.875419,Spanish Restaurant
4,PALACIO,40.40958,-3.88009,Trattoria Piccolo Mio,40.405704,-3.876092,Italian Restaurant


In [30]:
madrid_onehot = pd.get_dummies(madrid_venues[['venue Category']], prefix="", prefix_sep="")
madrid_onehot

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,...,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
madrid_onehot.insert(loc=0, column='neighborhood', value=madrid_venues['neighborhood'] )
madrid_onehot

,neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7245,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7246,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7247,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7248,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
madrid_grouped = madrid_onehot.groupby('neighborhood').mean().reset_index()
madrid_grouped = pd.drop()

,neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ABRANTES,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,ACACIAS,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,ADELFAS,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,AEROPUERTO,0.022727,0.0,0.022727,0.022727,0.022727,0.045455,0.045455,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,ALAMEDA DE OSUNA,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.023256,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,VENTAS,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
127,VILLAVERDE ALTO C.H.,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
128,VINATEROS,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
129,VISTA ALEGRE,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [35]:
#function returning categories for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
# get top ten common venues for each Neighborhood
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = madrid_grouped['neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,ABRANTES,Bar,Grocery Store,Gym / Fitness Center,Supermarket,Tapas Restaurant,Concert Hall,Bakery,Fast Food Restaurant,Plaza,...,Seafood Restaurant,Colombian Restaurant,Pizza Place,Breakfast Spot,Rock Club,Brewery,Coffee Shop,Burger Joint,Nightclub,Spanish Restaurant
1,ACACIAS,Restaurant,Tapas Restaurant,Spanish Restaurant,Art Gallery,Plaza,Market,Bar,Garden,Beer Garden,...,Art Museum,Bakery,Chinese Restaurant,Farmers Market,Burger Joint,Indie Theater,Breakfast Spot,Falafel Restaurant,Argentinian Restaurant,Coffee Shop
2,ADELFAS,Spanish Restaurant,Museum,Italian Restaurant,Bar,Food & Drink Shop,Market,Bakery,Peruvian Restaurant,Farmers Market,...,Greek Restaurant,Mexican Restaurant,Supermarket,Tapas Restaurant,Restaurant,Café,Soccer Field,Bookstore,Eastern European Restaurant,Hotel
3,AEROPUERTO,Hotel,Rental Car Location,Spanish Restaurant,Duty-free Shop,Airport Service,Airport Terminal,Metro Station,Coffee Shop,Grocery Store,...,Fast Food Restaurant,Food Court,Border Crossing,Bookstore,Snack Place,Bakery,Accessories Store,Smoke Shop,Italian Restaurant,Pizza Place
4,ALAMEDA DE OSUNA,Spanish Restaurant,Hotel,Hotel Bar,Plaza,Restaurant,Chinese Restaurant,Bar,Pharmacy,Tapas Restaurant,...,Garden,Bistro,Metro Station,Mexican Restaurant,Café,Food & Drink Shop,Smoke Shop,Scenic Lookout,Coffee Shop,Pub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,VENTAS,Spanish Restaurant,Restaurant,Bar,Chinese Restaurant,Hotel,Argentinian Restaurant,Pizza Place,Farmers Market,Burger Joint,...,Supermarket,Bakery,Convenience Store,Gastropub,Grocery Store,Italian Restaurant,Gym / Fitness Center,Pub,Japanese Restaurant,Mediterranean Restaurant
127,VILLAVERDE ALTO C.H.,Train Station,Grocery Store,Skate Park,Brewery,Spanish Restaurant,Pizza Place,Diner,Park,Gastropub,...,Train,Mobile Phone Shop,Furniture / Home Store,Exhibit,Fabric Shop,Yoga Studio,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market
128,VINATEROS,Pizza Place,Soccer Field,Coffee Shop,Grocery Store,Fast Food Restaurant,Bar,Supermarket,Restaurant,Garden,...,Big Box Store,Athletics & Sports,Park,Trail,Shopping Mall,Café,Historic Site,Deli / Bodega,Pool,Chinese Restaurant
129,VISTA ALEGRE,Bar,Grocery Store,Gym / Fitness Center,Supermarket,Tapas Restaurant,Concert Hall,Bakery,Fast Food Restaurant,Plaza,...,Seafood Restaurant,Colombian Restaurant,Pizza Place,Breakfast Spot,Rock Club,Brewery,Coffee Shop,Burger Joint,Nightclub,Spanish Restaurant


In [48]:
# Clustering for Neighborhood

# set number of clusters
kclusters = 5

madrid_grouped_clustering = madrid_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 3, 3, 0, 3, 0, 0, 3], dtype=int32)

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = boroughs_mad


In [51]:
# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')

madrid_merged.head()

,borough code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,1,CENTRO,PALACIO,40.40958,-3.88009,3,Spanish Restaurant,Fast Food Restaurant,Restaurant,Italian Restaurant,...,Skate Park,Smoke Shop,Department Store,Japanese Restaurant,Burger Joint,Irish Pub,Pizza Place,Mediterranean Restaurant,Escape Room,Farmers Market
1,1,CENTRO,EMBAJADORES,40.39152,-3.69289,0,Spanish Restaurant,Restaurant,Bar,Coffee Shop,...,Supermarket,Bakery,Farmers Market,Flea Market,Burger Joint,Pub,Clothing Store,Beer Garden,Gym,Plaza
2,1,CENTRO,CORTES,40.41607,-3.69893,3,Restaurant,Plaza,Hotel,Spanish Restaurant,...,Argentinian Restaurant,Theater,Cosmetics Shop,Pizza Place,Beer Store,Movie Theater,Fountain,Spa,Seafood Restaurant,Casino
3,1,CENTRO,JUSTICIA,40.42479,-3.69308,3,Restaurant,Hotel,Spanish Restaurant,Plaza,...,Japanese Restaurant,Brewery,Bistro,Market,Bakery,Theater,Bookstore,Art Museum,Clothing Store,Cocktail Bar
4,1,CENTRO,UNIVERSIDAD,40.42565,-3.70726,0,Café,Bookstore,Hotel,Tapas Restaurant,...,Coffee Shop,Peruvian Restaurant,Breakfast Spot,Theater,Miscellaneous Shop,Mexican Restaurant,Burger Joint,Gastropub,Wine Bar,Hostel


In [52]:
neighborhoods_venues_sorted.head()

,Cluster Labels,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,0,ABRANTES,Bar,Grocery Store,Gym / Fitness Center,Supermarket,Tapas Restaurant,Concert Hall,Bakery,Fast Food Restaurant,...,Seafood Restaurant,Colombian Restaurant,Pizza Place,Breakfast Spot,Rock Club,Brewery,Coffee Shop,Burger Joint,Nightclub,Spanish Restaurant
1,0,ACACIAS,Restaurant,Tapas Restaurant,Spanish Restaurant,Art Gallery,Plaza,Market,Bar,Garden,...,Art Museum,Bakery,Chinese Restaurant,Farmers Market,Burger Joint,Indie Theater,Breakfast Spot,Falafel Restaurant,Argentinian Restaurant,Coffee Shop
2,0,ADELFAS,Spanish Restaurant,Museum,Italian Restaurant,Bar,Food & Drink Shop,Market,Bakery,Peruvian Restaurant,...,Greek Restaurant,Mexican Restaurant,Supermarket,Tapas Restaurant,Restaurant,Café,Soccer Field,Bookstore,Eastern European Restaurant,Hotel
3,3,AEROPUERTO,Hotel,Rental Car Location,Spanish Restaurant,Duty-free Shop,Airport Service,Airport Terminal,Metro Station,Coffee Shop,...,Fast Food Restaurant,Food Court,Border Crossing,Bookstore,Snack Place,Bakery,Accessories Store,Smoke Shop,Italian Restaurant,Pizza Place
4,3,ALAMEDA DE OSUNA,Spanish Restaurant,Hotel,Hotel Bar,Plaza,Restaurant,Chinese Restaurant,Bar,Pharmacy,...,Garden,Bistro,Metro Station,Mexican Restaurant,Café,Food & Drink Shop,Smoke Shop,Scenic Lookout,Coffee Shop,Pub


In [53]:
#searching location for 'madrid, españa'

address = 'madrid, españa'

geolocator = Nominatim(user_agent="mad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


In [55]:
# Map with clusters for 'Madrid, España'
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters